In [1]:
import numpy
npmax = numpy.max
npmin = numpy.min
from numpy import float32, pi, imag, real, mean, maximum, nanmean, nanstd, absolute

from bokeh.plotting import Session, show, figure, output_server, hplot
from bokeh.models import Range1d
from bokeh.charts import Histogram

from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs
from gnss.tracking import tracking, CN0Algorithm
from gnss.visualization import TrackingPlot, TrackingMultiPlot, CorrelationGridPlot, \
    CorrelationLinePlot, FinePhasePlot, sample_histogram

In [2]:
!du -hs /mnt/gluster/by-drives/WesternDigital/WMC5D0D6NF5D/20150424_122600_G26/HI_A1_20150424_122600_004634_GPSL1_USRP1.sc4

14G	/mnt/gluster/by-drives/WesternDigital/WMC5D0D6NF5D/20150424_122600_G26/HI_A1_20150424_122600_004634_GPSL1_USRP1.sc4


In [3]:
filepath = '/mnt/gluster/by-drives/WesternDigital/WMC5D0D6NF5D/20150424_122600_G26/HI_A1_20150424_122600_004634_GPSL5_USRP5.sc4'
source = sources.FileSource(filepath, file_f_samp=25e6, f_center=1.17645e9, buffer_size=12.5e6, bit_depth=4, real=False, decimation=2)
source.i_msb = False  # this file has real/imaginary components flipped

In [4]:
session = Session(root_url='http://192.168.3.137:5006/', load_from_config=False)
session.login('anon', '1234')
output_server('hawaii', session=session)

In [5]:
# fig = sample_histogram(source.buffer)
# show(fig)

In [6]:
c_acq_plot = CorrelationLinePlot()
f_acq_plot = FinePhasePlot()
show(hplot(c_acq_plot.plot, f_acq_plot.plot))

In [14]:
step = 10e-3
cn0_algo = CN0Algorithm()
f_update = 1. / step
block_duration = 10e-3
block_size = block_duration * source.f_samp - 1
output_buff_size = source.buffer_size / (step * source.f_samp)

In [15]:
i_corr_plot = TrackingPlot('i corr', output_buff_size)
q_corr_plot = TrackingPlot('q corr', output_buff_size)
phase_plot = TrackingPlot('phase error', output_buff_size)
show(hplot(i_corr_plot.plot, q_corr_plot.plot, phase_plot.plot))
buffer_size = round(source.file_duration)
doppler_plot = TrackingPlot('doppler 1Hz', buffer_size)
cn0_plot = TrackingPlot('cn0 1Hz', buffer_size)
show(hplot(doppler_plot.plot, cn0_plot.plot))

In [9]:
source.seek(time=150)
source.load()
print('we have {0} seconds of data starting at {1} seconds'.format(source.buffer_size / source.f_samp, source.buffer_start_time))

we have 1.0 seconds of data starting at 150.0 seconds


In [10]:
svid = 26
signal = Signal.GPSL5_I(svid)

c_acq = coarse.CoarseAcquirerLowMem(source, 20e-3, 1, dopp_min=-5000, dopp_max=5000)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [11]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp0 = c_acq.f_dopp
time = c_acq.time
cn0 = c_acq.cn0
print(chip0, f_dopp0, time, cn0)
c_acq_plot.update(c_acq)

71471.6904 -1750.0 150.0 68.7723334785


In [12]:
f_acq.acquire(signal, time, chip0, f_dopp0)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)
f_acq_plot.update(f_acq)

71471.6904 -1776.59606237 150.0
0.0 0.056 1.20780774086 -0.962958635816


In [16]:
dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
# pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 32, zeta=1.7, fs=f_update)
fll_filter = iir_filter.FirstOrderLowpass(1 / pi, f_update);  fll_filter.f(0);  fll_filter.f(0)

delay = .5
correlator = tracking.Correlator(chip_delays=[-2 * delay, -delay, 0., delay, 2 * delay])

In [17]:
time = f_acq.time
theta = last_phase_error = 0.
chip = chip_out = chip0
f_dopp = f_acq.f_dopp
source.seek(time)
source.load()

In [ ]:
while True:
    while time < source.max_time - block_duration:
        code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
        earlier, early, prompt, late, later = code_corr
        chip_error = tracking.delay_discriminator(earlier, prompt, later)
        phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
        chip_error = dll_filter.f(chip_error)
        phase_error = pll_filter.f(phase_error)
        dopp_error = (phase_error - last_phase_error) / (step * 2 * pi)
        last_phase_error = phase_error
        dopp_error = fll_filter.f(dopp_error)
        # error correction
        chip -= chip_error
        theta -= phase_error
#         f_dopp += dopp_error
        # carrier-aiding and propagation
        time += step
        f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
        chip += step * f_chip
        f_inter = signal.f_carrier - source.f_center
        theta += step * (f_inter + f_dopp) * 2. * pi
        # outputs
        chip_out = chip_out - chip_error
        i_corr_plot.push_data(time, real(prompt))
        q_corr_plot.push_data(time, imag(prompt))
        phase_plot.push_data(time, phase_error)
#         print(f_dopp)
    data = phase_plot.data.get()
    f_dopp -= mean(data) / (2 * pi) / step
    print(f_dopp)
    i_corr_plot.update()
    q_corr_plot.update()
    phase_plot.update()
    cn0 = cn0_algo.compute(i_corr_plot.data.get(), q_corr_plot.data.get(), step)
    doppler_plot.push_data(time, f_dopp)
    cn0_plot.push_data(time, cn0)
    doppler_plot.update()
    cn0_plot.update()
    source.seek(time)
    source.load()

-1777.57119631
-1777.5989124
-1777.37728579
-1777.14337073
-1776.93468211
-1776.68887181
-1776.50456433
-1776.25093498
-1776.06388378
-1775.83809193